In [1]:
import yaml
from pathlib import Path

# only generate the citations for papers that have this person as an author
TARGET_AUTHOR = "lex"
# the students of the target author
STUDENTS = {}
# other values include paper | preprint | poster | thesis | commentary 
PUBLICATION_TYPES = {"paper"}


pub_dir = Path("../_publications")
paper_yaml = []

for file_path in pub_dir.iterdir():
    if not file_path.is_file():
        continue
    print(f"Processing file: {file_path.name}")
    with file_path.open("r", encoding="utf-8") as f:
        try:
            docs = yaml.safe_load_all(f)        # generator over all YAML documents in the stream
            first_doc = next(docs, None)       # get the first document (the Jekyll front matter)
            paper_yaml.append(first_doc)
        except Exception as e:
            print(f"Error reading {file_path.name}: {e}")
            paper_yaml.append(None)
            
# yaml_objects now contains the parsed front-matter object (or None on error)


Processing file: 2023_chi_misleading.md
Processing file: 2025_thesis_lisnic.md
Processing file: 2020_infovis_insights.md
Processing file: 2024_vis_aardvark.md
Processing file: 2025_chi_guardrails.md
Processing file: 2013_biovis_compariseq.md
Processing file: 2015_infovis_weaver.md
Processing file: 2011_infovis_visbricks.md
Processing file: 2012_infovis_dsm.md
Processing file: 2017_vds_coresets.md
Processing file: 2018_infovis_origraph.md
Processing file: 2012_thesis_bigelow.md
Processing file: 2012_biovis_enroute.md
Processing file: 2021_ivi_intent.md
Processing file: 2020_thesis_quinan.md
Processing file: 2023_nsf_revisit.md
Processing file: 2019_TBD_coresets.md
Processing file: .DS_Store
Error reading .DS_Store: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
Processing file: 2014_vast_contour.md
Processing file: 2021_ivi_sanguine.md
Processing file: 2023_thesis_rogers.md
Processing file: 2024_vis_loops.md
Processing file: 2014_vizsec_cyber-dashboard.md
Proc

In [2]:
def getOrderNumber(order_str):
    if type(order_str) == str:
        return int(order_str.split('-')[1])
    return order_str
    

def sortAndFilterPapers(paper_yaml, targetAuthor=TARGET_AUTHOR, publicationTypes=PUBLICATION_TYPES):
    """ 
    papers are filtered to only include those with targetAuthor in authors and of type in publicationTypes 
    papers are sorted by year descending, breaking ties by order field if it exists
    """
    
    filteredPapers = []
    for paper in paper_yaml:
        try:
             authors = set(paper.get("authors", []))
        except Exception as e:
            continue;       
       
        if targetAuthor in authors and paper["type"] in publicationTypes:
            filteredPapers.append(paper)
    # Sort papers by "year" in descending order, break ties by field "order" if it exists
    sorted_papers = sorted(filteredPapers, key=lambda x: (x.get("year", 0), getOrderNumber(x.get("order", 0))), reverse=True)
    return sorted_papers
print(len(paper_yaml))
paper_yaml = sortAndFilterPapers(paper_yaml, TARGET_AUTHOR)
print(len(paper_yaml))

148
68


Rules for authors: 
* the TARGET_AUTHOR is set in bold
* the students of the TARGET_AUTHOR are underlined

In [3]:
author_dir = Path("../_persons")

TYPES_OF_STUDENTS = ["phd_students", "ms_students", "ug_students", "phd_graduates", "former_students"]

def getStudentsOfAuthor(authorKey = TARGET_AUTHOR):
    """returns a set of author keys who are students of the given authorKey"""
    students = set()
    author_path = Path(author_dir) / f"{authorKey}.md"
    with author_path.open("r", encoding="utf-8") as f:
        try:
            docs = yaml.safe_load_all(f)        # generator over all YAML documents in the stream
            author_yaml = next(docs, None)
            for student_type in TYPES_OF_STUDENTS:
                student_list = author_yaml.get(student_type, [])
                if student_list is not None:
                    student_list = [s for s in student_list if " " not in s]
                    students.update(student_list)

        except Exception as e:
            print(f"Error reading {author_path.name}: {e}")
            return students
    return students


def resolveAuthors(authors):
    """takes a list of author keys or author names and returns a comma-separated string of author names followed by a period."""
    resolvedAuthors = ""

    # Authors can be keys (e.g., "lex") or full names (e.g., "John Doe"). Keys never have a space. 
    for author in authors:
        if (" ") in author:            
            resolvedAuthors += author + ", "
        else:
            authorKey = author
            author_path = Path(author_dir) / f"{authorKey}.md"
            # if not author_path.is_file():
            #     print(f"Error: Author file {author_path} does not exist.")
            #     continue
            
            with author_path.open("r", encoding="utf-8") as f:
                try:
                    docs = yaml.safe_load_all(f)        # generator over all YAML documents in the stream
                    author_yaml = next(docs, None)       # get the first document (the Jekyll front matter)
                    author_name = f"{author_yaml.get("first_name")} {author_yaml.get("last_name")}"
                    if authorKey == TARGET_AUTHOR:
                        author_name = f"\\textbf{{{author_name}}}"
                    if authorKey in STUDENTS: 
                        author_name = f"\\underline{{{author_name}}}"
                    resolvedAuthors += author_name + ", "
                except Exception as e:
                    print(f"Error reading {author_path.name}: {e}")
    
    return resolvedAuthors[:-2]+". "

STUDENTS = getStudentsOfAuthor(TARGET_AUTHOR)
print(STUDENTS)
resolveAuthors(["lex", "meyer", "John Doe", "zcutler"])


{'murali', 'waller', 'anirudh', 'sahai', 'luke', 'nobre', 'kiran', 'lisnic', 'lin', 'cherkaev', 'wootton', 'rogers', 'zcutler', 'abrar', 'sunny', 'kern', 'eliza', 'lange', 'mengjiao', 'hardasani', 'ssiu', 'zoexel', 'mehrpour', 'prajan', 'mukherjee', 'pranav', 'aljuhani', 'ilkin', 'hannah', 'marno'}


'\\textbf{Alexander Lex}, Miriah Meyer, John Doe, \\underline{Zach Cutler}. '

The goal:

```latex
\item \underline{Maxim Lisnic}, \underline{Zach Cutler}, Marina Kogan, \textbf{Alexander Lex}, \textit{Visualization Guardrails: Designing Interventions Against Cherry-Picking in Interactive Data Explorers}, SIGCHI Conference on Human Factors in Computing Systems (CHI),  pp.\ 1-19, \href{http://dx.doi.org/10.1145/3706598.3713385}{doi:10.1145/3706598.3713385}, 2025. \textbf{Best Paper Award}
```

```latex
\item \underline{Devin Lange}, Robert Judson-Torres, Thomas A. Zangle, \textbf{Alexander Lex}, \textit{Aardvark: Composite Visualizations of Trees, Time-Series, and Images}, IEEE Transactions on Visualization and Computer Graphics (VIS), vol.\ 31, no.\ 1, pp.\ 1290–1300, \url{http://dx.doi.org/10.31219/osf.io/cdbm6}, 2024. \textbf{Best Paper Award.}
```

Bibentry for journal papers: 

```yaml
bibentry: article
bib:
    journal: IEEE Transactions on Visualization and Computer Graphics (InfoVis)
    booktitle:
    editor:
    publisher: IEEE
    address:
    doi: 10.1109/TVCG.2020.3030405
    url: 
    volume: 27
    number: 2
    pages: 1106-1116
    month: Feb.
```

Bibentry for conference papers:
```yaml
bibentry: inproceedings
bib:
  booktitle: "SIGCHI Conference on Human Factors in Computing Systems (CHI)"
  publisher: ACM
  doi: 10.1145/3544548.3580910
  pages: 1-21
```

In [4]:
# we allow article | inproceedings | phdthesis | book, but only doing article/inproceedings for now

inproceedings="inproceedings"
article="article"

def generateCitation(paper):
    authors = resolveAuthors(paper.get("authors"))

    ### TITLE 
    title = paper["title"]+". "
    #print(paper)
  
    ### YEAR
    year = str(paper["year"])+". "
  
    ### DOI
    try:
        doi = paper["bib"]["doi"]
        if doi is None:
            doi = ""
        else:    
            # generate  \href{http://dx.doi.org/10.1145/3706598.3713385}{doi:10.1145/3706598.3713385}, 
            doi = f"\\doi{{{doi}}}, "
    except KeyError:
        doi = ""
    
    ### PAGES
    pages = paper["bib"].get("pages", "") 
    if pages == None:
        pages = ""
    if pages != "":
        pages = f"pp.\\ {pages}, "

    ### VENUE
    venue = ""
    if(paper["bibentry"] == inproceedings):
        booktitle = paper["bib"].get("booktitle", "")
        publisher = paper["bib"].get("publisher", "")  
        if publisher is None:
            publisher = ""
        venue  = f"{publisher} {booktitle}, "  
    
    elif(paper["bibentry"] == article):       
        try:
            journal = paper["bib"].get("journal", "")+", "   
            venue = journal
        except Exception as e:
            print(paper)
            print("Error")
            return 
        
    volume = paper["bib"].get("volume", "") 
    if volume == None:
        volume = ""
    if volume != "":    
        volume = f"vol.\\ {volume}, "

    number = paper["bib"].get("number", "")     
    if number == None:     
        number = ""
    if number != "":
        number = f"no.\\ {number}, "

    award = paper.get("award", "")
    if award is None:
        award = ""
    if award != "":
        award = f"\\textbf{{{award}}}"
 
    return f"\\item {authors}{title}{venue}{volume}{number}{pages}{doi}{year}{award}"

         
    




print("final: " + generateCitation(paper_yaml[65]))
print("final: " + generateCitation(paper_yaml[1]))
print("final: " + generateCitation(paper_yaml[2]))
print("final: " + generateCitation(paper_yaml[6]))


final: \item Marc Streit, \textbf{Alexander Lex}, Heimo Müller, Dieter Schmalstieg. Gaze-Based Focus Adaption in an Information Visualization System.  Proceedings of the Conference on Computer Graphics and Visualization and Image Processing (CGVCVIP), 2009. 
final: \item \underline{Haihan Lin}, \underline{Maxim Lisnic}, Derya Akbaba, Miriah Meyer, \textbf{Alexander Lex}. Here’s what you need to know about my data: Exploring Expert Knowledge’s Role in Data Analysis. IEEE Transactions on Visualization and Computer Graphics (VIS), vol.\ 32, no.\ 1, \doi{10.1109/TVCG.2025.3634821}, 2026. \textbf{IEEE VIS 2025 Honorable Mention Award}
final: \item Tingying He, Maggie McCracken, Daniel Hajas, Sarah Creem-Regehr, \textbf{Alexander Lex}. Using Tactile Charts to Support Comprehension and Learning of Complex Visualizations for Blind and Low-Vision Individuals. IEEE Transactions on Visualization and Computer Graphics (VIS), vol.\ 32, 2026. 
final: \item \underline{Zach Cutler}, Lane Harrison, \un

In [5]:
papers = []
count = 1
for paper in paper_yaml:
   citation = generateCitation(paper)
   papers.append(citation)
   print(count, citation)
   count += 1

# write papers to file  
with open("publications.tex", "w", encoding="utf-8") as f:
    for citation in papers:
        f.write(citation + "\n")

1 \item \underline{Zach Cutler}, Jack Wilburn, Hilson Shrestha, Yiren Ding, Brian Bollen, \underline{Khandaker Abrar Nadib}, Tingying He, Andrew McNutt, Lane Harrison, \textbf{Alexander Lex}. ReVISit 2: A Full Experiment Life Cycle User Study Framework. IEEE Transactions on Visualization and Computer Graphics (VIS), vol.\ 32, 2026. \textbf{IEEE VIS 2025 Best Paper Award}
2 \item \underline{Haihan Lin}, \underline{Maxim Lisnic}, Derya Akbaba, Miriah Meyer, \textbf{Alexander Lex}. Here’s what you need to know about my data: Exploring Expert Knowledge’s Role in Data Analysis. IEEE Transactions on Visualization and Computer Graphics (VIS), vol.\ 32, no.\ 1, \doi{10.1109/TVCG.2025.3634821}, 2026. \textbf{IEEE VIS 2025 Honorable Mention Award}
3 \item Tingying He, Maggie McCracken, Daniel Hajas, Sarah Creem-Regehr, \textbf{Alexander Lex}. Using Tactile Charts to Support Comprehension and Learning of Complex Visualizations for Blind and Low-Vision Individuals. IEEE Transactions on Visualizati